![image](https://drive.google.com/u/0/uc?id=15DUc09hFGqR8qcpYiN1OajRNaASmiL6d&export=download)

# **Taller No. 1 - ISIS4825**
## **Proceso de Aprendizaje Automático e Introducción a la Clasificación**
## **Contenido**
1. [**Objetivos**](#id1)
2. [**Problema**](#id2)
3. [**Importando las librerías necesarias para el laboratorio**](#id3)
4. [**Visualización y Análisis Exploratorio**](#id4)
5. [**Preparación de los Datos**](#id5)
6. [**Modelamiento**](#id6)
7. [**Predicción**](#id7)
8. [**Validación**](#id8)

## **Objetivos**<a name="id1"></a>
- Familiarizarse con las librerías de Scikit-Learn y con el algoritmo de KNN
- Resolver un problema de clasificación multiclase y tomar métricas de desempeño sobre este
## **Problema**<a name="id2"></a>

## **Importando las librerías necesarias para el laboratorio**<a name="id3"></a>

In [ ]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.metrics import (precision_score, recall_score, confusion_matrix, 
                             accuracy_score, f1_score, roc_curve)

import utils.general as gen

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns

## **Visualización y Análisis Exploratorio**<a name="id4"></a>
- Vamos a hacer uso del Dataset `Fashion-MNIST` que consta de 10 clases:
    0. T-Shirt/Top
    1. Trouser
    2. Pullover
    3. Dress
    4. Coat
    5. Sandal
    6. Shirt 
    7. Sneaker
    8. Bag
    9. Ankle Boot
- De igual forma, el dataset tiene 70.000 imágenes en escala de rises con resolución 28x28. Sin embargo, las imágenes ya se encuentran aplanadas con tamaño ed vector 784.

In [ ]:
fashion_mnist = datasets.fetch_openml("Fashion-MNIST")

In [ ]:
data, target = fashion_mnist.data, fashion_mnist.target

In [ ]:
data.shape, target.shape

In [ ]:
random_sample = np.random.choice(np.arange(len(data)), 9)
gen.visualize_subplot(
    data[random_sample].reshape(-1, 28, 28),
    target[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(data)), 9)
gen.visualize_subplot(
    data[random_sample].reshape(-1, 28, 28),
    target[random_sample],  (3, 3), (6, 6)
)

In [ ]:
target_classes = ["T-Shirt/Top", "Trouser", "Pullover", "Dress", 
                  "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

In [ ]:
target_distribution = pd.Series(target).value_counts().sort_index()
target_distribution.index = target_classes

In [ ]:
target_distribution

## **Preparación de los Datos**<a name="id5"></a>
- Dado que estamos trabajando con modelos de Machine Learning superficial, vamos a necesitar que todas nuestras imágenes sean convertidas a vectores, si es que aún no lo son.

### **Tratamiento de Imágenes**

In [ ]:
sample_img = data[0].reshape(28, 28)
sample_target = target[0]

In [ ]:
gen.imshow(sample_img, color=False)

In [ ]:
sample_img.shape

In [ ]:
sample_img = sample_img.flatten()

In [ ]:
sample_img.shape

### **Train Set, Validation Set, Test Set**
- Generalmente, en el mundo del computer vision, se hace la siguiente partición de datasets:
    - Train Data:
        - Train Set
        - Validation Set
    - Test Data:
        - Test Set
- La partición de los datasets la podemos hacer de varias formas, pero en esta ocasión veremos la partición por índices y por contenido.
#### **Partición por Índice**

In [ ]:
ss_full_train_test = ShuffleSplit(n_splits=10, test_size=10000, random_state=1234)

In [ ]:
for full_train_index, test_index in ss_full_train_test.split(data):
    pass

In [ ]:
full_train_index

In [ ]:
test_index

In [ ]:
full_train_set, test_set = ((data[full_train_index], target[full_train_index]), 
                            (data[test_index], target[test_index]))

In [ ]:
ss_train_val = ShuffleSplit(n_splits=10, test_size=10000, random_state=5678)

In [ ]:
for train_index, val_index in ss_train_val.split(full_train_set[0]):
    pass

In [ ]:
train_set, val_set = ((full_train_set[0][train_index], full_train_set[1][train_index]), 
                      (full_train_set[0][val_index], full_train_set[1][val_index]))

In [ ]:
X_train, y_train = train_set[0], train_set[1]
X_val, y_val = val_set[0], val_set[1]
X_test, y_test = test_set[0], test_set[1]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_val.shape, y_val.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
random_sample = np.random.choice(np.arange(len(X_train)), 9)
gen.visualize_subplot(
    X_train[random_sample].reshape(-1, 28, 28),
    y_train[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_val)), 9)
gen.visualize_subplot(
    X_val[random_sample].reshape(-1, 28, 28),
    y_val[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_test)), 9)
gen.visualize_subplot(
    X_test[random_sample].reshape(-1, 28, 28),
    y_test[random_sample],  (3, 3), (6, 6)
)

#### **Partición por Contenido**

In [ ]:
full_X_train, X_test, full_y_train, y_test = train_test_split(data, target, 
                                                              test_size=10000, 
                                                              random_state=1234)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(full_X_train, full_y_train, 
                                                  test_size=10000, 
                                                  random_state=1234)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_train)), 9)
gen.visualize_subplot(
    X_train[random_sample].reshape(-1, 28, 28),
    y_train[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_val)), 9)
gen.visualize_subplot(
    X_val[random_sample].reshape(-1, 28, 28),
    y_val[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_test)), 9)
gen.visualize_subplot(
    X_test[random_sample].reshape(-1, 28, 28),
    y_test[random_sample],  (3, 3), (6, 6)
)

## **Modelamiento**<a name="id6"></a>
### **K-Nearest-Neighbors**

In [ ]:
knn_clf = KNeighborsClassifier()

In [ ]:
knn_clf.fit(X_train, y_train)

## **Predicción**<a name="id7"></a>

In [ ]:
pred = knn_clf.pred(X_val[:1000])

## **Validación**<a name="id8"></a>
- En esta etapa de evaluación realizamos el proceso de toma de métricas. Por lo tanto, dado que estamos resolviendo un problema de clasificación, vamos a usar la matriz de confusión.
- Vamos a calcular la precisión, la cobertura y el puntaje F1.
- Precision: $\frac{TP}{TP + FP}$
- Cobertura: $\frac{TP}{TP + FN}$ (Recall, Sensitivity)
- Accuracy score: $\frac{TP + TN}{TP + TN + FP + FN}$
- F1 score: $\frac{TP}{TP + \frac{FN + FP}{2}}$ (Harmonic Mean)